In [5]:
import pandas as pd
import re
def unicode(match):
    match = match.group()
    return chr(int(match[2:], base=16))
list_csv = ['data/kor/attraction_review',
            'data/kor/hotel_review',
            'data/kor/restaurant_review',
            'data/eng/eng_attraction_review',
            'data/eng/eng_hotel_review',
            'data/eng/eng_restaurant_review']

In [6]:
for csv in list_csv:
    df = pd.read_csv(csv+'.csv',
                         names=['placeId','userId','score','title','review'],
                         encoding='euc-kr')
    print('na',len(df[df['userId'].isna()]),'개 ',end='')
    count = 0
    for i in df['userId']:
        if not pd.isna(i) and (re.fullmatch(r' +', i) != None):
            count+=1
    print('공백', count,'개')

na 0 개 공백 25 개
na 0 개 공백 2 개
na 7120 개 공백 0 개
na 934 개 공백 0 개
na 0 개 공백 4 개
na 2730 개 공백 0 개


In [9]:
##For user-based CF
for csv in list_csv:
    df_raw = pd.read_csv(csv+'.csv',
                         names=['placeId','userId','score','title','review'],
                         encoding='ms949')
    df_raw = df_raw.drop(df_raw.columns[3:],axis=1)
#replace space to NaN when cell only have space
    df_raw['placeId'] = df_raw['placeId'].replace('^ +$', float('nan'), regex=True)
    df_raw['userId'] = df_raw['userId'].replace('^ +$', float('nan'), regex=True)
    df_raw['score'] = df_raw['score'].replace('^ +$', float('nan'), regex=True)
#drop rows with NaN in placeId, userId, score
    before = len(df_raw)
    df_raw = df_raw.dropna(subset=['placeId','userId','score'])
    after = len(df_raw)
    print(before - after, 'entries droped in ', csv)
    df_raw = df_raw.sort_values(by=['userId']) 
#rescale score as 1~5
    df_raw['score'] = df_raw['score']/10
    df_raw['score'] = df_raw['score'].apply(int)
#unicode replacement
    for i in df_raw['userId']:
        i = re.sub(r'\\u[0-9a-f]{4}', unicode, i)
    df_raw['userId'] = df_raw['userId'].apply(lambda x:
    re.sub(r'\\u[0-9a-f]{4}', unicode, x))
#reset index and save
    df = df_raw.reset_index(drop=True)
    df.to_csv(csv+'_user.csv',mode='w')

25 entries droped in  data/kor/attraction_review
2 entries droped in  data/kor/hotel_review
7120 entries droped in  data/kor/restaurant_review
934 entries droped in  data/eng/eng_attraction_review
4 entries droped in  data/eng/eng_hotel_review
2730 entries droped in  data/eng/eng_restaurant_review


In [4]:
##For tag-based CF
for csv in list_csv:
    df_raw = pd.read_csv(csv+'.csv',
                         names=['placeId','userId','score','title','review'],
                         encoding='ms949')
    df_raw = df_raw.drop(df_raw.columns[5:],axis=1)
#merge title into review
    df_raw['review'] = df_raw.title + ' ' + df_raw['review']
    df_raw = df_raw.drop(columns=['title'])
#replace space to NaN when cell only have space
    df_raw['placeId'] = df_raw['placeId'].replace('^ +$', float('nan'), regex=True)
    df_raw['userId'] = df_raw['userId'].replace('^ +$', float('nan'), regex=True)
    df_raw['score'] = df_raw['score'].replace('^ +$', float('nan'), regex=True)
#drop rows with NaN in placeId, userId, score
    before = len(df_raw)
    df_raw = df_raw.dropna(subset=['placeId','userId','score'])
    after = len(df_raw)
    print(before - after, 'entries droped in ', csv)
    df_raw = df_raw.sort_values(by=['userId']) 
#remove html tags
    try:
        df_raw['review'] = df_raw['review'].apply(lambda x: re.sub('<br/>',' ',x))
    except:
        print(df_raw)
#replace multiple spaces into one
    df_raw['review'] = df_raw['review'].apply(lambda x: re.sub(' +',' ',x))
#rescale score as 1~5
    df_raw['score'] = df_raw['score']/10
    df_raw['score'] = df_raw['score'].apply(int)
#unicode replacement
    for i in df_raw['userId']:
        i = re.sub(r'\\u[0-9a-f]{4}', unicode, i)
    df_raw['userId'] = df_raw['userId'].apply(lambda x:
    re.sub(r'\\u[0-9a-f]{4}', unicode, x))
#reset index and save
    df = df_raw.reset_index(drop=True)
    df.to_csv(csv+'_user.csv',mode='w')

25 entries droped in  data/kor/attraction_review
2 entries droped in  data/kor/hotel_review
7120 entries droped in  data/kor/restaurant_review
        placeId                userId  score                          title  \
25069   4075310   \uc131\ud6c8 \uc870     50                  역시 모모야마 최고네요~   
25041   4075310   \uc131\ud6c8 \uc870     50                  롯데호텔 모모야마 ^ ^   
35727   7748139   \uc740\uc8fc \uae40     50                 양 조절할 수 있고 좋아요   
22572   4050286   \uc740\uc8fc \uae40     40                    생선까스 먹으러 가요   
30715   6208125   \uc740\uc8fc \uae40     30                           맛있어요   
34948   7618195             ok hyun K     50             향토 맥주를 즐기기 좋은 따뜻한    
52805   9094211           -Max_Park-1     50   족발 맛집으로 유명해서 방문했는데 정말 맛집입니다.   
8654    1892430             007\ube75     40         진짜 맛있는 함흥냉면을 먹을 수 있는 곳   
48898   9008441             007\ube75     50             시원한 팥라떼는 최고의 건강음료~   
4199    1196214             007\ube75     50              명동오면 생각나는

TypeError: expected string or bytes-like object